In [4]:
import matplotlib.pyplot as plt
import numpy as np
from graphtime import GroupFusedGraphLasso, IndepFusedGraphLasso
from graphtime.utils import get_edges, get_change_points, plot_data_with_cps
from graphtime.utils import precision, recall, f1_score, grid_search

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 7.0)

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
y = np.load('data/y.npy')
sigma = np.load('data/sigma.npy')
sigma_inv = np.load('data/sigma_inv.npy')
T = 90 # Steps
K = 2 # Changepoints
P = 10 # Variables
M = 5 # Active Edges
eps = 0.000001 # Edge threshold epsilon

In [9]:
edges = get_edges(sigma_inv[0], eps, P)
y_true = get_change_points(sigma_inv, eps, T, P)

### Run Estimation with GFGL

In [10]:
verbose = False
tol = 1e-4
max_iter = 500
gammas = [1, 1, 1] # gamma_V1, gamma_V2, gamma_W
lambda1 = 0.15
lambda2 = 25
gfgl = GroupFusedGraphLasso(lambda1, lambda2, gammas[0], gammas[1], gammas[2], tol, max_iter, verbose)
gfgl.fit(y);

### F, Precision, Recall
According to [this paper](https://projecteuclid.org/download/pdfview_1/euclid.ejs/1351865118), we treat graph
recovery as a classification problem of edges. For the mathematical definition, see paper p.19.

In [15]:
precision(gfgl.sparse_Theta, sigma_inv, per_ts=False)

0.11178451178451154

In [16]:
recall(gfgl.sparse_Theta, sigma_inv, per_ts=False)

1.0

In [18]:
f1_score(gfgl.sparse_Theta, sigma_inv, per_ts=False)

0.20109024833434244

### Exhaustive Grid Search with performance estimation

In [21]:
lams = np.logspace(0, 11, base=2, num=10)
pfs = grid_search(GroupFusedGraphLasso, y, sigma_inv, lams, lams)

In [22]:
f1s = {v[2]:k for k,v in pfs.items()}
prc = {v[0]:k for k,v in pfs.items()}
rec = {v[1]:k for k,v in pfs.items()}

In [23]:
max(prc.keys())

0.6666666666666666

In [24]:
f1s[0.5238095238095243]

(1.0, 2048.0)